In [2]:
import yfinance as yf

# Download stock data for Apple (AAPL) for the past 5 years
data = yf.download('AAPL', start='2018-01-01', end='2023-01-01')

# Display the first few rows of the data
print(data.head())

[*********************100%***********************]  1 of 1 completed

Price                      Adj Close      Close       High        Low  \
Ticker                          AAPL       AAPL       AAPL       AAPL   
Date                                                                    
2018-01-02 00:00:00+00:00  40.524342  43.064999  43.075001  42.314999   
2018-01-03 00:00:00+00:00  40.517292  43.057499  43.637501  42.990002   
2018-01-04 00:00:00+00:00  40.705486  43.257500  43.367500  43.020000   
2018-01-05 00:00:00+00:00  41.168930  43.750000  43.842499  43.262501   
2018-01-08 00:00:00+00:00  41.016022  43.587502  43.902500  43.482498   

Price                           Open     Volume  
Ticker                          AAPL       AAPL  
Date                                             
2018-01-02 00:00:00+00:00  42.540001  102223600  
2018-01-03 00:00:00+00:00  43.132500  118071600  
2018-01-04 00:00:00+00:00  43.134998   89738400  
2018-01-05 00:00:00+00:00  43.360001   94640000  
2018-01-08 00:00:00+00:00  43.587502   82271200  


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Use the 'Close' price for prediction
prices = data['Close'].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
prices = scaler.fit_transform(prices.reshape(-1, 1))

# Create sequences of data for RNN input
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 60  # Use the last 60 days to predict the next day
X, y = create_sequences(prices, sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

rnn_model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dense(1)
])

rnn_model.compile(optimizer='adam', loss='mean_absolute_error')

In [5]:
# Reshape X_train and X_test for RNN input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

rnn_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10

60/60 [==============================] - 2s 12ms/step - loss: 0.0741 - val_loss: 0.0875
Epoch 2/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0219 - val_loss: 0.0384
Epoch 3/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0161 - val_loss: 0.0368
Epoch 4/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0152 - val_loss: 0.0307
Epoch 5/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0152 - val_loss: 0.0347
Epoch 6/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0149 - val_loss: 0.0331
Epoch 7/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0155 - val_loss: 0.0288
Epoch 8/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0145 - val_loss: 0.0410
Epoch 9/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0141 - val_loss: 0.0322
Epoch 10/10
60/60 [==============================] - 1s 9ms/step - loss: 0.0139 - val_loss: 0.0272


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Flatten the training and test data
X_train_lr = X_train.reshape((X_train.shape[0], -1))
X_test_lr = X_test.reshape((X_test.shape[0], -1))

# Define and train the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train_lr, y_train)

LinearRegression()

In [7]:
# Evaluate RNN model
rnn_predictions = rnn_model.predict(X_test)
rnn_mae = mean_absolute_error(y_test, rnn_predictions)

# Evaluate Linear Regression model
lr_predictions = linear_model.predict(X_test_lr)
lr_mae = mean_absolute_error(y_test, lr_predictions)

# Print the results
print(f"RNN Model MAE: {rnn_mae}")
print(f"Linear Regression Model MAE: {lr_mae}")

8/8 [==============================] - 0s 4ms/step
RNN Model MAE: 0.027150507152291673
Linear Regression Model MAE: 0.019825266386765957
